In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# Load the merged dataset
merged_data = pd.read_csv("merged_data.csv")
merged_data

,Transaction_ID,Company_ID,Product_ID,Quantity,Transaction_Date,Product_Price_x,Total_Cost,Product_Name,Product_Price_y,Company_Name,Company_Profit,Address
0,1.0,88.0,6.0,11.0,2024-03-26,194379.147964,1075200.0,RevenueVue Dashboard,179200.0,Elite Consulting 88,75950.0,"EDSA, Barangay 456, Taguig, Philippines"
1,2.0,29.0,19.0,16.0,2021-02-21,97930.993380,1428000.0,EcoNomix Modeler,95200.0,Sky Industries 29,61952.0,"Edsa, brgy. 606, makati, philippines!"
2,4995.5,28.0,18.0,6.0,2021-02-21,126095.547778,940800.0,DashSync Analytics Hub,134400.0,Quantum Associates 67,75301.5,"Katipunan Ave, Barangay 123, Mandaluyong, Phil..."
3,4.0,85.0,12.0,12.0,2021-02-21,131600.000000,1008000.0,BudgetMaster Pro,84000.0,Green Ventures 85,113470.0,"EDSA, Barangay 707, Cebu City, Philippines"
4,5.0,47.0,3.0,8.0,2021-02-21,99575.609634,705600.0,TrendWise Forecaster,100800.0,Green Industries 47,31130.0,"Taft Ave, Barangay 707, Mandaluyong, Philippines"
...,...,...,...,...,...,...,...,...,...,...,...,...
10995,9997.0,39.0,2.0,9.0,2021-05-17,159518.597391,1512000.0,MarketMinder Analytics,168000.0,Prime Consulting 39,107255.0,"Ortigas Ave, Barangay 606, Davao City, Philipp..."
10996,4995.5,90.0,1.0,15.0,2021-02-21,128137.094759,1960000.0,FinPredictor Suite,140000.0,Prime Holdings 90,46225.0,"EDSA, Barangay 202, Parañaque, Philippines"
10997,9999.0,33.0,NaN,19.0,2021-04-15,81786.119894,1680000.0,RiskRadar Monitor,151200.0,Green Consulting 33,52003.0,"Shaw blvd, barangay 707, mandaluyong, philippi..."
10998,9999.0,33.0,NaN,19.0,2021-04-15,81786.119894,1680000.0,SpendTrack Plus,106400.0,Green Consulting 33,52003.0,"Shaw blvd, barangay 707, mandaluyong, philippi..."


> ###Feature Engineering###

In [7]:
# Adjusting the OneHotEncoder initialization to remove the unsupported 'sparse_output' argument
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

if categorical_columns_to_encode:
    encoded_features = pd.DataFrame(
        encoder.fit_transform(merged_data[categorical_columns_to_encode]),
        columns=encoder.get_feature_names_out(categorical_columns_to_encode)
    )

    # Add encoded features to the dataset
    merged_data = pd.concat([merged_data.reset_index(drop=True), encoded_features.reset_index(drop=True)], axis=1)

    # Drop original categorical columns after encoding
    merged_data = merged_data.drop(columns=categorical_columns_to_encode)
else:
    print("No valid categorical columns found for one-hot encoding.")

In [9]:
# Define numerical columns for scaling (ensure they exist)
numerical_columns = [
    col for col in ['Product_Price', 'Quantity', 'Total_Cost', 'Company_Profit']
    if col in merged_data.columns
]

if numerical_columns:
    scaler = StandardScaler()
    merged_data[numerical_columns] = scaler.fit_transform(merged_data[numerical_columns])
else:
    print("No valid numerical columns found for scaling.")

# Final Check: Ensure all features are processed
print("Feature Engineering Complete. Dataset Preview:")
merged_data

Feature Engineering Complete. Dataset Preview:


,Transaction_ID,Company_ID,Product_ID,Quantity,Transaction_Date,Product_Price_x,Total_Cost,Product_Price_y,Company_Profit,Product_Name_BudgetMaster Pro,...,"Address_Shaw blvd, barangay 707, mandaluyong, philippines!","Address_Slex, barangay 123, pasig, philippines","Address_Slex, barangay 404, taguig, philippines","Address_Slex, barangay 505, manila, philippines","Address_Taft Ave, Barangay 123, Manila, Philippines!","Address_Taft Ave, Barangay 456, Cebu City, Philippines","Address_Taft Ave, Barangay 606, Makati, Philippines!","Address_Taft Ave, Barangay 707, Mandaluyong, Philippines","Address_Taft ave, barangay 202, baguio, philippines","Address_Taft ave, brgy. 456, manila, philippines"
0,1.0,88.0,6.0,0.079475,2024-03-26,194379.147964,-0.393619,179200.0,-0.007179,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,29.0,19.0,0.985783,2021-02-21,97930.993380,0.016924,95200.0,-0.625512,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4995.5,28.0,18.0,-0.826833,2021-02-21,126095.547778,-0.550016,134400.0,-0.035825,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,85.0,12.0,0.260737,2021-02-21,131600.000000,-0.471817,84000.0,1.650190,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,47.0,3.0,-0.464310,2021-02-21,99575.609634,-0.823711,100800.0,-1.987011,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,9997.0,39.0,2.0,-0.283048,2021-05-17,159518.597391,0.114672,168000.0,1.375655,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10996,4995.5,90.0,1.0,0.804522,2021-02-21,128137.094759,0.635996,140000.0,-1.320220,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10997,9999.0,33.0,NaN,1.529568,2021-04-15,81786.119894,0.310169,151200.0,-1.064989,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10998,9999.0,33.0,NaN,1.529568,2021-04-15,81786.119894,0.310169,106400.0,-1.064989,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Define X (features) and y (target)
X = merged_data.drop(columns=['Product_ID'])
y = merged_data['Product_ID']

# Convert target variable to categorical (if necessary)
y = y.astype(str)

### Split Data ###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

> #Model 1:Multinomial Logistic Regression

In [19]:
# Handle missing values by imputing or dropping rows with NaN values
# Dropping rows with NaN values for simplicity
X_train = X_train.dropna()
X_test = X_test.dropna()
y_train = y_train[X_train.index]
y_test = y_test[X_test.index]

# Retry training the model
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42, max_iter=1000)
logistic_model.fit(X_train, y_train)

# Predictions
y_pred_logistic = logistic_model.predict(X_test)

# Evaluation Metrics
logistic_metrics = {
    "Model": "Multinomial Logistic Regression",
    "Accuracy": accuracy_score(y_test, y_pred_logistic),
    "Classification Report": classification_report(y_test, y_pred_logistic, output_dict=True)
}

# Print results
print("Multinomial Logistic Regression Metrics:")
print(f"Accuracy: {logistic_metrics['Accuracy']}")
print(classification_report(y_test, y_pred_logistic))

# Save predictions to a DataFrame
logistic_predictions = pd.DataFrame({"Actual": y_test, "Predicted": y_pred_logistic})
logistic_predictions.to_csv("logistic_predictions.csv", index=False)

/root/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/root/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

> #Model 2: Random Forest Classifier

In [21]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predictions
y_pred_rf = rf_model.predict(X_test)

# Evaluation Metrics
rf_metrics = {
    "Model": "Random Forest Classifier",
    "Accuracy": accuracy_score(y_test, y_pred_rf),
    "Classification Report": classification_report(y_test, y_pred_rf, output_dict=True)
}

# Print results
print("Random Forest Classifier Metrics:")
print(f"Accuracy: {rf_metrics['Accuracy']}")
print(classification_report(y_test, y_pred_rf))

# Save predictions to a DataFrame
rf_predictions = pd.DataFrame({"Actual": y_test, "Predicted": y_pred_rf})
rf_predictions.to_csv("rf_predictions.csv", index=False)

Random Forest Classifier Metrics:
Accuracy: 0.9969309462915601
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        80
        10.0       1.00      1.00      1.00        74
        11.0       1.00      1.00      1.00        96
        12.0       1.00      1.00      1.00        76
        13.0       1.00      1.00      1.00        74
        14.0       1.00      1.00      1.00        77
        15.0       1.00      1.00      1.00        67
        16.0       0.97      0.95      0.96        77
        17.0       1.00      1.00      1.00        78
        18.0       1.00      1.00      1.00        76
        19.0       1.00      1.00      1.00        77
         2.0       1.00      1.00      1.00        89
        20.0       1.00      1.00      1.00        86
         3.0       1.00      1.00      1.00        86
         4.0       1.00      1.00      1.00        85
         5.0       1.00      1.00      1.00        76
         6.0      

The Random Forest Classifier achieved an impressive accuracy of 99.69%. The classification report shows high precision, recall, and F1-scores across all classes.

> #Model Comparison Matrix
# Consolidate metrics into a single table

In [23]:
comparison_table = pd.DataFrame({
    "Model": [logistic_metrics["Model"], rf_metrics["Model"]],
    "Accuracy": [logistic_metrics["Accuracy"], rf_metrics["Accuracy"]],
    "Precision (Weighted Avg)": [
        logistic_metrics["Classification Report"]["weighted avg"]["precision"],
        rf_metrics["Classification Report"]["weighted avg"]["precision"]
    ],
    "Recall (Weighted Avg)": [
        logistic_metrics["Classification Report"]["weighted avg"]["recall"],
        rf_metrics["Classification Report"]["weighted avg"]["recall"]
    ],
    "F1-Score (Weighted Avg)": [
        logistic_metrics["Classification Report"]["weighted avg"]["f1-score"],
        rf_metrics["Classification Report"]["weighted avg"]["f1-score"]
    ]
})

# Save comparison table to CSV
comparison_table.to_csv("model_comparison.csv", index=False)

# Print the comparison table
comparison_table

,Model,Accuracy,Precision (Weighted Avg),Recall (Weighted Avg),F1-Score (Weighted Avg)
0,Multinomial Logistic Regression,0.165217,0.057132,0.165217,0.078226
1,Random Forest Classifier,0.996931,0.996962,0.996931,0.996933


> # Predict on the same sample data

In [25]:
sample_data = X_test.iloc[:12]  # Predicting 12 datapoints
logistic_predictions = logistic_model.predict(sample_data)
rf_predictions = rf_model.predict(sample_data)

# Combine predictions into a single DataFrame
comparison_df = pd.DataFrame({
    'Sample Index': sample_data.index,
    'Actual': y_test.loc[sample_data.index].values,
    'Logistic Regression Prediction': logistic_predictions,
    'Random Forest Prediction': rf_predictions
})

# Display the comparison table
comparison_df

,Sample Index,Actual,Logistic Regression Prediction,Random Forest Prediction
0,5484,18.0,nan,18.0
1,6998,6.0,nan,6.0
2,3984,6.0,nan,6.0
3,3111,13.0,20.0,13.0
4,4040,16.0,16.0,16.0
5,3013,11.0,20.0,11.0
6,6607,12.0,nan,12.0
7,4219,5.0,nan,5.0
8,8750,6.0,nan,6.0
9,5665,2.0,20.0,2.0


> from sklearn.metrics import accuracy_score, classification_report

In [27]:
# Predict probabilities for the test set
rf_pred_probs = rf_model.predict_proba(X_test)
logistic_pred_probs = logistic_model.predict_proba(X_test)

# --- Printing the predictions side by side ---
# Create a DataFrame for the predictions to print them side by side
logistic_pred_df = pd.DataFrame(logistic_pred_probs, columns=[f'Product_{i}' for i in range(1, len(logistic_pred_probs[0]) + 1)])
rf_pred_df = pd.DataFrame(rf_pred_probs, columns=[f'Product_{i}' for i in range(1, len(rf_pred_probs[0]) + 1)])

# Print the predicted probabilities from both models side by side
comparison_df = pd.concat([logistic_pred_df, rf_pred_df], axis=1, keys=["Logistic Regression", "Random Forest"])

# --- Accuracy and Metrics ---
# Predict classes for evaluation (select the product with the highest probability)
logistic_preds = logistic_model.predict(X_test)
rf_preds = rf_model.predict(X_test)

# Calculate accuracy scores
logistic_accuracy = accuracy_score(y_test, logistic_preds)
rf_accuracy = accuracy_score(y_test, rf_preds)

# --- Display Classification Reports ---
logistic_report = classification_report(y_test, logistic_preds)
rf_report = classification_report(y_test, rf_preds)

comparison_df, logistic_accuracy, rf_accuracy, logistic_report, rf_report

/root/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(     Logistic Regression                                                    \
                Product_1 Product_2 Product_3 Product_4 Product_5 Product_6   
 0               0.054122  0.022402  0.055334  0.014823  0.052164  0.044020   
 1               0.045466  0.024349  0.043462  0.015034  0.026087  0.047610   
 2               0.046247  0.010974  0.064421  0.005381  0.060447  0.045903   
 3               0.037231  0.002749  0.070597  0.001038  0.134580  0.025190   
 4               0.048948  0.011178  0.063383  0.006447  0.091758  0.037348   
 ...                  ...       ...       ...       ...       ...       ...   
 1950            0.046916  0.006666  0.063094  0.003321  0.101689  0.028126   
 1951            0.045756  0.029529  0.046843  0.020407  0.029885  0.049481   
 1952            0.043746  0.041388  0.035342  0.034066  0.020352  0.040940   
 1953            0.032098  0.003450  0.044331  0.001803  0.121441  0.016727   
 1954            0.044081  0.056247  0.029076  0.052

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ab2b7b4f-3b27-4d02-8095-1a9b5599a04f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>